<a href="https://colab.research.google.com/github/G-Gaddu/Quant-Connect/blob/main/Kelly%20Criterion%20using%20SMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import the required packages
from AlgorithmImports import *

class SMA_with_KellyCriterion(QCAlgorithm):

    def initialize(self):
        # Set the start and end dates
        self.set_start_date(2015, 1, 1)
        self.set_end_date(2025, 1, 1)
        # Set strategy cash
        self.set_cash("USD", 1000000)
        # Let the risky asset be Tesla and the risk free asset a US government bond fund
        # Limit the leverage to 5 times
        self._risky_asset = self.add_equity('TSLA', Resolution.HOUR, leverage=5)
        self._risk_free_asset = self.add_equity('SHY', Resolution.HOUR, leverage=5)
        # Add short and long term sma indicators for the risky asset
        self._risky_asset.short_sma = self.sma(self._risky_asset.symbol, 1)
        self._risky_asset.long_sma = self.sma(self._risky_asset.symbol, 6)
        # Call up the Kelly Criterion
        self._risky_asset.signal = 0
        self._kc = KellyCriterion(1.5, 40)
        # Add a warm-up period
        self.set_warm_up(timedelta(365))
        # Record the exposure to Tesla each day after trading
        self._tsla_weights = []
        ticker = Symbol.create('SPY', SecurityType.EQUITY, Market.USA)
        self.schedule.on(self.date_rules.every_day(self._risky_asset.symbol), self.time_rules.after_market_close(ticker), self._record_weight)

    def on_end_of_algorithm(self):
        # Determine the average weight in Tesla stocks at the end
        self.log(f"Average weight: {sum(self._tsla_weights) / len(self._tsla_weights)}")

    def _record_weight(self):
        # Create a function to record the weight of Tesla at the end of trading
        self._tsla_weights.append(self._risky_asset.holdings.holdings_value / self.portfolio.total_portfolio_value)

    def on_data(self, data: Slice):
        # If the market is not open then stop
        if not self.is_market_open(self._risky_asset.symbol) or not data.bars:
            return
        # Feed the signal into the Kelly Criterion
        if self._risky_asset.short_sma < self._risky_asset.long_sma and self._risky_asset.signal:
            self._risky_asset.signal = 0
            self._kc.update_signal(0, self._risky_asset.price)
        elif self._risky_asset.short_sma > self._risky_asset.long_sma and not self._risky_asset.signal:
            self._risky_asset.signal = 1
            self._kc.update_signal(1, self._risky_asset.price)

        # If we are in the warmup period or the Kelly Criterion is not ready then stop
        if self.is_warming_up or not self._kc.is_ready:
            return

        # Assign weights based on the signal
        if self._risky_asset.holdings.is_long and not self._risky_asset.signal:
            # If the signal is 0 then assign everything to the bond fund
            self.set_holdings([PortfolioTarget(self._risk_free_asset.symbol, 1)], True)
        # Otherwise invest in Tesla but limit exposure to 600%
        elif not self._risky_asset.holdings.is_long and self._risky_asset.signal:
            weight = min(6, self._kc.weight())
            self.set_holdings(
                [PortfolioTarget(self._risky_asset.symbol, weight),
                    # If the weight is less than 100%, the remainder is invested in the bond fund
                    PortfolioTarget(self._risk_free_asset.symbol, 0 if weight > 1 else 1-weight)
                ]
            )

class KellyCriterion:

    def __init__(self, factor, period):
        self._factor = factor
        self._period = period
        self._trades = np.array([])

    def weight(self):
        # First ensure that there are enough trades to make calculations with
        if not self.is_ready:
            return None
        # Determine the Kelly percentage
        gains = self._trades[self._trades > 0]
        losses = self._trades[self._trades < 0]
        if not gains.sum():
            return 0
        if not losses.sum():
            return self._factor
        # Determine the metrics
        gain_loss_ratio = gains.mean() / losses.mean()
        winning_probability = len(gains) / self._period
        # Return the weight calculcated using the Kelly Criterion
        return self._factor*(winning_probability - (1-winning_probability)/gain_loss_ratio)

    def update_signal(self, signal, price):
        # Checks updated signal, if the signal stops then exit the position
        if not signal:
            self._trades = np.append(self._trades, [price - self._entry_price])[-self._period:]
        # Otherwise continue with the updated price
        else:
            self._entry_price = price

    def is_ready(self):
        # Check if enough trades have been executed over the time period
        return len(self._trades) == self._period